In [ ]:
import numpy as np
import pandas as pd
import sys
sys.path.append("../scripts")
import pandas as pd
if 'chatbot' in sys.modules:
    del sys.modules['chatbot']
from chatbot import Chatbot
from tqdm.notebook import trange

In [ ]:
def prompt_label_sentences(text, labels):
    return f"""Example1:
Labels:
{{'1': 'Pieces: 1.0, Manufacturer: Danfoss Redan, SubType: NaN, HxType: Isoleret varmeveksler, NominalEffectEach: NaN, Year: NaN', '2': 'Pieces: NaN, Manufacturer: NaN, SubType: NaN, HxType: NaN, NominalEffectEach: NaN, Year: NaN', '3': 'Pieces: NaN, Manufacturer: NaN, SubType: NaN, HxType: NaN, NominalEffectEach: NaN, Year: NaN', '4': 'Pieces: NaN, Manufacturer: NaN, SubType: NaN, HxType: NaN, NominalEffectEach: NaN, Year: NaN'}}
Sentences:
['Fjernvarme med isoleret veksler (indirekte anlæg) - nyere', ' Ejendommen opvarmes med fjernvarme fra HOFOR', '\n\nTil opvarmning af radiatorerne er der 1 stk', ' isoleret varmevekslere monteret i fjernvarmeunit,  fabrikat Redan', '\n\nFjernvarmeunit er placeret i kælderen', '']
Output:
['1', '0', '1', '1', '0']

Example2
Labels:
{{'1': 'Pieces: 1.0, Manufacturer: Unknown, SubType: NaN, HxType: Isoleret varmeveksler, NominalEffectEach: NaN, Year: NaN', '2': 'Pieces: 1.0, Manufacturer: Reci, SubType: LP280H-46, HxType: Isoleret varmeveksler, NominalEffectEach: NaN, Year: 2016', '3': 'Pieces: 1.0, Manufacturer: Reci, SubType: LP120-60 , HxType: Isoleret varmeveksler, NominalEffectEach: NaN, Year: 2017.0', '4': 'Pieces: 1.0, Manufacturer: Armatech, SubType: Cvcu-p SO14404 , HxType: Isoleret varmeveksler, NominalEffectEach: NaN, Year: 2018.0'}}
Sentences:
['Fjernvarme - isoleret veksler - indirekte - nyere anlæg', ' Bygningen opvarmes med fjernvarme', ' Anlægget er udført med 4 isolerede varmeveksler og indirekte centralvarmevand i fordelingsnettet', ' Alle vekslerne er placeret i varmecentralen i kælderen under bygning 1', ' \n\nVeksler 1 er af ukendt fabrikat, type og årgang', ' \nVeksler 2 er af fabrikat og type, RECI LP280H-46 og er fra 2016', ' \nVeksler 3 er af fabrikat og type, RECI LP120-60 og er fra 2017', ' \n\nDer er desuden en veksler 4 som forsyner udendørsbanerne', ' Den er af fabrikat og type, Armatech Cvcu-p SO14404 og fra 2018', ' Denne er ikke en del af beregning for energimærket', ' ']
Output:
['0', '0', 'multiple', 'multiple', '1', '2', '3', '4', '4', '0', '0']

TASK:
You are an expert in energy labels and Danish language.
You are given a list of sentences, you are also given a list for 4 exchangers. For each sentence, you must determine if it contains a reference to one of the exchagers, and if so, which one. Thus the output is a python list of labels (i.e. strings), one for each sentence. 
The labels are:
- '0' for no reference to any exchanger
- '1' for reference to exchanger 1
- '2' for reference to exchanger 2
- '3' for reference to exchanger 3
- '4' for reference to exchanger 4
- 'multiple' for reference to more than one exchanger

Labels:
{labels}

Sentences:
{text}

YOUR TASK: Return a Python list of labels and ONLY a list of labels, one for each sentence.

Output:
"""

In [3]:
MODEL_PATH = "../models/Phi-3.1-mini-4k-instruct-Q5_K_M.gguf"
N_CTX = 4096
N_THREADS = 7
chatbot = Chatbot(MODEL_PATH, context_size=N_CTX, n_threads=N_THREADS)

ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_set_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_c4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_id_bf16_f16                (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h64           (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h80           (not supported)
ggml

In [4]:
d_gt = pd.read_excel("../data/data_district_heating.xlsx", sheet_name="Ground Truth")
d_gt['text'] = d_gt['S_text'].fillna("").astype(str) + ". " + d_gt['L_text'].fillna("").astype(str)

In [5]:
def labels_to_string(data, i):
    columns = ['Pieces', 'Manufacturer', 'SubType', 'HxType', 'NominalEffectEach', 'Year']

    one = data.iloc[i, 7:13].fillna("NaN").astype(str).tolist()
    two = data.iloc[i, 13:19].fillna("NaN").astype(str).tolist()
    three = data.iloc[i, 19:25].fillna("NaN").astype(str).tolist()
    four = data.iloc[i, 25:31].fillna("NaN").astype(str).tolist()

    one = [f"{col}: {val}" for col, val in zip(columns, one)]
    two = [f"{col}: {val}" for col, val in zip(columns, two)]
    three = [f"{col}: {val}" for col, val in zip(columns, three)]
    four = [f"{col}: {val}" for col, val in zip(columns, four)]
    return {
        "1": ", ".join(one),
        "2": ", ".join(two),
        "3": ", ".join(three),
        "4": ", ".join(four)
    }

In [6]:
pbar = trange(len(d_gt), desc="Processing samples", leave=True)
res = []
errors = []
for i in pbar:
    row = {}
    row['text']= d_gt.loc[i, 'text']
    row['sentences'] = row['text'].split('.')
    row['labels'] = labels_to_string(d_gt, i)
    prompt = prompt_label_sentences(row['sentences'], row['labels'])
    try:
        row['response'] = await chatbot.prompt(prompt, max_tokens=100, stop=["]"])
        res.append(row)
        pbar.set_postfix({'response': row['response']})
    except Exception as e:
        print(f"Error processing sample {i}: {e}")
        errors.append(i)

Processing samples:   0%|          | 0/2272 [00:00<?, ?it/s]

In [8]:
print(res[4]['sentences'])
print(res[4]['labels'])
print(res[4]['response'])

['Fjernvarme med isoleret veksler (indirekte anlæg) - udført før 1970', ' Bygningen opvarmes med fjernvarme', ' Anlægget er udført med isoleret varmeveksler og indirekte centralvarmevand i fordelingsnettet', '\n']
{'1': 'Pieces: 1.0, Manufacturer: Unknown, SubType: NaN, HxType: Isoleret varmeveksler, NominalEffectEach: NaN, Year: Before 1970', '2': 'Pieces: NaN, Manufacturer: NaN, SubType: NaN, HxType: NaN, NominalEffectEach: NaN, Year: NaN', '3': 'Pieces: NaN, Manufacturer: NaN, SubType: NaN, HxType: NaN, NominalEffectEach: NaN, Year: NaN', '4': 'Pieces: NaN, Manufacturer: NaN, SubType: NaN, HxType: NaN, NominalEffectEach: NaN, Year: NaN'}
['1', '0', 'multiple'


In [9]:
len(res)

2272

In [11]:
res_df = pd.DataFrame(res)
res_df.to_csv("../outputs/sentences_labels_ground_truths.csv")

In [20]:
import ast

labels = []
errors = []
for i in range(len(res)):
    try:
        list = ast.literal_eval(res[i]['response'] + "]")
        labels.append(list)
    except Exception as e:
        errors.append((i, str(e)))
        labels.append([])

print(labels)

[['0', '0', 'multiple', '0'], ['0', '0'], ['1', '0', '1'], ['1', '0', '1', '1', '0'], ['1', '0', 'multiple'], ['0', '0', '1'], ['1', '0', '1', '3', '4', '4'], ['1', '0', '0', '1', '0', '0', '1', '1'], ['1', '0'], ['1', '0'], ['0', '0', 'multiple'], ['1', '0'], ['1', '0', 'multiple', '0', '0'], ['1', '0'], ['0', '0', 'multiple'], ['1', '0'], ['0', '0'], ['0', '0', '0'], ['1', '0', 'multiple', '2', '0'], ['1', '0'], ['1', '0'], ['1', '0', '0'], ['1', '0', '0'], ['1', '0', '1', '0', 'multiple'], ['1', '0', '1', '0', '0', '0'], ['1', '0', '1', '1', '2'], ['1', '0', 'multiple', '2', '0'], ['1', '0', '1', '0'], ['1', '0', '1', '0', 'multiple'], ['1', '0', '1', '0', '0', '0'], ['1', '0', '1', '1', '2'], ['1', '0', '1', 'multiple', '0'], ['1', '0', '1', '1', '0'], ['1', '0', '1'], ['1', '0', '0'], ['0', '0', '1', '0'], ['1', '0', '1', '0'], ['1', '0', '0', '0', '1', 'multiple', 'multiple'], ['1', '0', '0'], ['1', '0', '0'], ['1', '0', '0'], ['1', '0', '0'], ['1', '0'], ['1', '0', '0'], ['1', '

In [35]:
different_lengths = []
for i in range(len(res)):
    sentences = [x.strip() for x in res[i]['sentences']]
    sentences_2 = sentences[:-1] if sentences[-1] == '' else sentences
    if len(labels[i]) != len(sentences) and len(labels[i]) != len(sentences_2):
        different_lengths.append(i)
print(len(different_lengths))

284


In [36]:
for i in different_lengths[:10]:
    print("---")
    print(f"len sentences {i}:", len(res[i]['sentences']))
    print(f"len labels {i}:", len(labels[i]))
    print(f"Sample {i}:", res[i]['sentences'])
    print(f"Labels {i}:", labels[i])

---
len sentences 12: 7
len labels 12: 5
Sample 12: ['Fjernvarme med isoleret veksler (indirekte anlæg) - efter 1980', ' Ejendommen opvarmes med fjernvarme', ' Anlægget er udført med isoleret varmeveksler og indirekte centralvarmevand i fordelingsnettet', '\n\nFra hovedrør i varmeventralen, fordeler rør ud til varmefordelingsanlægget blandesløjfer, der fordeler varme rundt til de enkelte bygningsdele/zoners radiatorer', ' \nBlandesløjfer er placeret i kælderen og er primært ved de enkelte ventilationsanlæg', ' \nFra hovedstrenge og blandesløjfer er der yderligere fordeling til ventilationsanlæggens varmeflader', '']
Labels 12: ['1', '0', 'multiple', '0', '0']
---
len sentences 73: 7
len labels 73: 5
Sample 73: ['Fjernvarmeveksler - København', ' Bygningen opvarmes med fjernvarme', ' Anlægget er udført med 2 stk', ' isolerede varmeveksler af typen Reci, årgang 1985', ' Vekslerne er placeret i fælles varmecentral, som er beliggende i ejendommen Amagerbrogade nr', ' 174', '']
Labels 73: [

In [ ]:
pbar = trange(len(different_lengths), desc="Processing samples", leave=True)
for i in pbar:
    row = {}
    idx = different_lengths[i]
    prompt = prompt_label_sentences(res[idx]['sentences'], res[idx]['labels'])
    try:
        res[idx]['response'] = await chatbot.prompt(prompt, max_tokens=100, stop=["]"])
        pbar.set_postfix({'response': res[idx]['response']})
    except Exception as e:
        print(f"Error processing sample {i}: {e}")
        errors.append(i)

Processing samples:   0%|          | 0/284 [00:00<?, ?it/s]